In [3]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt 250
length of features used = 500
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [4]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt 150
length of features used = 300
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [5]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_chosen_set.csv 9936
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_chosen_set.csv 9240
length of features used = 19176
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5003144654088051
cosine K-Nearest Neighbours
0.5044418238993711
euclidean K-Nearest Neighbours
0.502122641509434
braycurtis K-Nearest Neighbours
0.5029088050314465
canberra K-Nearest Neighbours
0.4995283018867924
jaccard K-Nearest Neighbours
0.5011399371069183


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.49772012578616354
************************************ ****************************


In [6]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt']
['[img]url;[/img]\n', 'smartart\n', 'oic\n', '/o\n', 'floodtech\n', '-2013.url\n', '-563-4131\n', 'tivoli\n', '', 'barbara\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt 100
['hoto\n', '', 'neal\n', 'ee\n', 'lich\n', '&#7883;ch\n', '443\n', 'sia\n', '[author]\n', 'mikayla\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt 100
length of features used = 200
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [7]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt']
['[img]url;[/img]\n', 'smartart\n', 'oic\n', '/o\n', 'floodtech\n', '-2013.url\n', '-563-4131\n', 'tivoli\n', '', 'barbara\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt 100
['hoto\n', '', 'neal\n', 'ee\n', 'lich\n', '&#7883;ch\n', '443\n', 'sia\n', '[author]\n', 'mikayla\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt 100
length of features used = 200
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [8]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt']
['[img]url;[/img]\n', 'smartart\n', 'oic\n', '/o\n', 'floodtech\n', '-2013.url\n', '-563-4131\n', 'tivoli\n', '', 'barbara\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt 150
['hoto\n', '', 'neal\n', 'ee\n', 'lich\n', '&#7883;ch\n', '443\n', 'sia\n', '[author]\n', 'mikayla\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt 150
length of features used = 300
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [9]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt']
['[img]url;[/img]\n', 'smartart\n', 'oic\n', '/o\n', 'floodtech\n', '-2013.url\n', '-563-4131\n', 'tivoli\n', '', 'barbara\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.1M.txt 250
['hoto\n', '', 'neal\n', 'ee\n', 'lich\n', '&#7883;ch\n', '443\n', 'sia\n', '[author]\n', 'mikayla\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013PMI.2F.txt 250
length of features used = 500
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [11]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt']
['i\n', 'u\n', 'and\n', '?\n', 'it\n', 'me\n', 'date\n', 'my\n', 'in\n', '???\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt 100
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt 100
length of features used = 200
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [12]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt']
['i\n', 'u\n', 'and\n', '?\n', 'it\n', 'me\n', 'date\n', 'my\n', 'in\n', '???\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt 150
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt 150
length of features used = 300
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [14]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt']
['i\n', 'u\n', 'and\n', '?\n', 'it\n', 'me\n', 'date\n', 'my\n', 'in\n', '???\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.1M.txt 250
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013ProbD.2F.txt 250
length of features used = 500
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [17]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt']
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt 250
length of features used = 200
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [18]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")










txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt']
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt 250
length of features used = 300
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440


KeyboardInterrupt: 

In [19]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:500]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt']
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013CHI.txt 250
length of features used = 500
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [20]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt']
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt 250
length of features used = 200
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [21]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt']
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt 250
length of features used = 300
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [22]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/PAN2013_Tokens/PAN2013_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2016/2016txtfiles/PAN2016_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        #reader = csv.reader(f, delimiter=",")
        #next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:500]
        print(word_list1[0:10])
        print(txt_file, len(word_list1))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
  #      next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt']
['you\n', 'a\n', '.\n', 'your\n', 'with\n', 'that\n', 'to\n', 'swinger\n', 'will\n', 'club\n']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013GSS.txt 250
length of features used = 500
len(all_training_text) 13106


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide


len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5
************************************ ****************************


In [23]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
5
manhattan K-Nearest Neighbours
0.4972877358490566
cosine K-Nearest Neighbours
0.5043238993710691
euclidean K-Nearest Neighbours
0.5036163522012579
braycurtis K-Nearest Neighbours
0.5038522012578617
canberra K-Nearest Neighbours
0.4995283018867924
jaccard K-Nearest Neighbours
0.5006682389937107


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.4970518867924528
************************************ ****************************


In [24]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=chi2, k=200)
5
manhattan K-Nearest Neighbours
0.5018474842767295
cosine K-Nearest Neighbours
0.49422169811320754
euclidean K-Nearest Neighbours
0.49996069182389935
braycurtis K-Nearest Neighbours
0.49897798742138366
canberra K-Nearest Neighbours
0.4979166666666667
jaccard K-Nearest Neighbours
0.49984276729559746


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.4979952830188679
************************************ ****************************


In [25]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=300)")
selector = SelectKBest(score_func=chi2, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=chi2, k=300)
5
manhattan K-Nearest Neighbours
0.5011399371069183
cosine K-Nearest Neighbours
0.5017295597484277
euclidean K-Nearest Neighbours
0.5041666666666667
braycurtis K-Nearest Neighbours
0.5053459119496856
canberra K-Nearest Neighbours
0.5009827044025157
jaccard K-Nearest Neighbours
0.5037735849056604


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5005110062893082
************************************ ****************************


In [26]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=500)")
selector = SelectKBest(score_func=chi2, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=chi2, k=500)
5
manhattan K-Nearest Neighbours
0.5050314465408805
cosine K-Nearest Neighbours
0.5032232704402516
euclidean K-Nearest Neighbours
0.5040880503144655
braycurtis K-Nearest Neighbours
0.5018867924528302
canberra K-Nearest Neighbours
0.5043632075471698
jaccard K-Nearest Neighbours
0.5031446540880503


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5006289308176101
************************************ ****************************


In [27]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=200)")
selector = SelectKBest(score_func=mutual_info_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=mutual_info_classif, k=200)
5
manhattan K-Nearest Neighbours
0.49547955974842767
cosine K-Nearest Neighbours
0.5007468553459119
euclidean K-Nearest Neighbours
0.4991745283018868
braycurtis K-Nearest Neighbours
0.4983490566037736
canberra K-Nearest Neighbours
0.49677672955974844
jaccard K-Nearest Neighbours
0.49897798742138366


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5026729559748427
************************************ ****************************


In [28]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=300)")
selector = SelectKBest(score_func=mutual_info_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=mutual_info_classif, k=300)
5
manhattan K-Nearest Neighbours
0.5023977987421384
cosine K-Nearest Neighbours
0.503066037735849
euclidean K-Nearest Neighbours
0.5031446540880503
braycurtis K-Nearest Neighbours
0.49783805031446543
canberra K-Nearest Neighbours
0.5012185534591195
jaccard K-Nearest Neighbours
0.5049135220125787


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.49901729559748426
************************************ ****************************


In [29]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
selector = SelectKBest(score_func=mutual_info_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=mutual_info_classif, k=500)
5
manhattan K-Nearest Neighbours
0.5018081761006289
cosine K-Nearest Neighbours
0.5022012578616353
euclidean K-Nearest Neighbours
0.5012971698113208
braycurtis K-Nearest Neighbours
0.4992138364779874
canberra K-Nearest Neighbours
0.4959119496855346
jaccard K-Nearest Neighbours
0.498938679245283


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5020440251572327
************************************ ****************************


In [30]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=f_classif, k=200)
5
manhattan K-Nearest Neighbours
0.5008254716981132
cosine K-Nearest Neighbours
0.5010220125786163
euclidean K-Nearest Neighbours
0.5057783018867924
braycurtis K-Nearest Neighbours
0.4995676100628931
canberra K-Nearest Neighbours
0.5032232704402516
jaccard K-Nearest Neighbours
0.5024764150943396


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5002751572327044
************************************ ****************************


In [31]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=300)")
selector = SelectKBest(score_func=f_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=f_classif, k=300)
5
manhattan K-Nearest Neighbours
0.5034984276729559
cosine K-Nearest Neighbours
0.4993317610062893
euclidean K-Nearest Neighbours
0.5011792452830188
braycurtis K-Nearest Neighbours
0.49669811320754714
canberra K-Nearest Neighbours
0.5011006289308176
jaccard K-Nearest Neighbours
0.4982311320754717


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.501375786163522
************************************ ****************************


In [32]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=500)")
selector = SelectKBest(score_func=f_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_chosen_set.1.csv 9721
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_chosen_set.1.csv 9145
length of features used = 18866
len(all_training_text) 13106
len(X_train)= 13106 len(y_train)= 13106
len(all_test_text) 25440
len(X_test= 25440 len(y_test)= 25440
selector = SelectKBest(score_func=f_classif, k=500)
5
manhattan K-Nearest Neighbours
0.5012971698113208
cosine K-Nearest Neighbours
0.5011006289308176
euclidean K-Nearest Neighbours
0.5007861635220126
braycurtis K-Nearest Neighbours
0.49504716981132074
canberra K-Nearest Neighbours
0.5022798742138365
jaccard K-Nearest Neighbours
0.49406446540880505


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5007468553459119
************************************ ****************************


In [ ]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=500)")
#selector = SelectKBest(score_func=chi2, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

In [ ]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2013/output2013OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2013/2013.1_textfiles/PAN2013_tweet_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        #reader = csv.reader(f, delimiter=",")
#        #next(reader) # skip header
#        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=500)")
#selector = SelectKBest(score_func=chi2, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

In [ ]:
# Module TermSelection.py processing
# -*- coding: utf-8 -*-

import math, codecs, string
import fnmatch


print "Read TermSelection.py"


def doAll(aFileM, aFileF, aYear="2013"):
   aFileName = "output" + aYear
   
   aL = doDifferenceSymetric("CHI", aFileM,  aFileF, aFileName+"CHI.txt", 550, 0)
   aL = doDifferenceSymetric("GSS", aFileM,  aFileF, aFileName+"GSS.txt", 550, 0)
   aL = doDifferenceSymetric("CC", aFileM, aFileF, aFileName+"CC.txt", 550, 0);
   aL = doDifferenceSymetric("GR", aFileM, aFileF, aFileName+"GR.txt", 550, 0);
   aL = doDifferenceSymetric("IG", aFileM, aFileF, aFileName+"IG.txt", 550, 0);
   
   aL = doDifferenceAsymetric("RS", aFileM, aFileF, aFileName+"RS.", 550, 0);
   aL = doDifferenceAsymetric("PMI", aFileM, aFileF, aFileName+"PMI.", 550, 0);
   aL = doDifferenceAsymetric("WLLR", aFileM, aFileF, aFileName+"WLLR.", 550, 0);
   aL = doDifferenceAsymetric("RF", aFileM,  aFileF, aFileName+"RF.", 550, 0)
   aL = doDifferenceAsymetric("OR", aFileM,  aFileF, aFileName+"OR.", 550, 0)
   return(True)


#
# Difference between two dictionaries
#

def doDifferenceProbD(aFileNameBase, aFileNameComp, outputFile="output.txt", aNumber=100, minDF=5):
   (aDictM, aNm) = createDictionary(aFileNameBase)
   print len(aDictM),
   if (minDF > 0):
      filterDictTF(aDictM, minDF)
      print len(aDictM)
   else:
      print
   aDictM = normalizeSimpleDict(aDictM)
   (aDictF, aNf) = createDictionary(aFileNameComp)
   print len(aDictF),
   if (minDF > 0):
      filterDictTF(aDictF, minDF)
      print len(aDictF)
   else:
      print
   aDictF = normalizeSimpleDict(aDictF)
   
   aD     = diffDictionary(aDictM, aDictF)
   if (aNumber > 0):
      aT1 = extractTopSortedDict(aD, aNumber, True)
      dumpDictionary(aT1, outputFile+"1M.txt")
   else:
      aT1 = aD
      dumpDictionaryAll(aD, outputFile+"1M.txt", True)
      
   aD = diffDictionary(aDictF, aDictM)
   if (aNumber > 0):
      aT2 = extractTopSortedDict(aD, aNumber, True)
      dumpDictionary(aT2, outputFile+"2F.txt")
   else:
      aT2 = aD
      dumpDictionaryAll(aD, outputFile+"2F.txt", True)      
   return(aT1, aT2)

#
# Difference with a given method
#
def doDifferenceAsymetric(aMethod, aFileNameBase, aFileNameComp, outputFile="outputOR.txt", aNumber=100, minDF=5):
   (aDictM, aNm) = createDFDictionary(aFileNameBase)
   print len(aDictM),
   if (minDF > 0):
      filterDictTF(aDictM, minDF)
      print len(aDictM)
   else:
      print
   (aDictF, aNf) = createDFDictionary(aFileNameComp)
   print len(aDictF),
   if (minDF > 0):
      filterDictTF(aDictF, minDF)
      print len(aDictF)
   else:
      print
   if (aMethod == "OR"):
      aD = computeOddRatio(aDictM, aDictF, aNm, aNf)
   if (aMethod == "PMI"):
      aD = computePMI(aDictM, aDictF, aNm, aNf)
   if (aMethod == "RF"):
      aD = computeRF(aDictM, aDictF, aNm, aNf)
   if (aMethod == "RS"):
      aD = computeRS(aDictM, aDictF, aNm, aNf)
   if (aMethod == "WLLR"):
      aD = computeWLLR(aDictM, aDictF, aNm, aNf)
      
   aT1 = extractTopSortedDict(aD, aNumber)
   dumpDictionary(aT1, outputFile+"1M.txt")
   if (aMethod == "OR"):
      aD = computeOddRatio(aDictF, aDictM, aNf, aNm)
   if (aMethod == "PMI"):
      aD = computePMI(aDictF, aDictM, aNf, aNm)
   if (aMethod == "RF"):
      aD = computeRF(aDictF, aDictM, aNf, aNm)
   if (aMethod == "RS"):
      aD = computeRS(aDictF, aDictM, aNf, aNm)
   if (aMethod == "WLLR"):
      aD = computeWLLR(aDictF, aDictM, aNf, aNm)
      
   aT2 = extractTopSortedDict(aD, aNumber)
   dumpDictionary(aT2, outputFile+"2F.txt")      
   return(aT1, aT2)

def doDifferenceSymetric(aMethod, aFileNameBase, aFileNameComp, outputFile="output.txt", aNumber=100, minDF=5):
   (aDictM, aNm) = createDFDictionary(aFileNameBase)
   print len(aDictM),
   if (minDF > 0):
      filterDictTF(aDictM, minDF)
      print len(aDictM)
   else:
      print
   (aDictF, aNf) = createDFDictionary(aFileNameComp)
   print len(aDictF),
   if (minDF > 0):
      filterDictTF(aDictF, minDF)
      print len(aDictF)
   else:
      print
   if (aMethod == "GSS"):
      aD = computeGSS(aDictM, aDictF, aNm, aNf)
   if (aMethod == "CHI"):
      aD = computeCHI(aDictM, aDictF, aNm, aNf)
   if (aMethod == "GR"):
      aD = computeGR(aDictM, aDictF, aNm, aNf)
   if (aMethod == "IG"):
      aD = computeIG(aDictM, aDictF, aNm, aNf)
   if (aMethod == "CC"):
      aD = computeCC(aDictM, aDictF, aNm, aNf)
   aDictF = aDictM = 0
   aB = extractBottomSortedDict(aD, aNumber)
   aT = extractTopSortedDict(aD, aNumber)
   dumpDictionary(aT, outputFile)
   dumpDictionary(aB, outputFile)
   return(aT, aB)

#
# Term Selection procedure 
#
#
# GSS  return a Dict (word:gss) from a corpusDFDict, subcorpusDFDict, 
def computeRF(aDict1DF, aDict2DF, subCorpusSize1, subCorpusSize2, aSet=[]):
   aDictValue = {}
   acValue = float(subCorpusSize1)
   bdValue = float(subCorpusSize2)
   nValue = acValue + bdValue
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0
      maxValue = max(bValue,1.0)
#     print aKey, aValue, bValue, maxValue, math.log (2 + (aValue/maxValue),2)
      aDictValue [aKey] =  math.log (2 + (aValue/maxValue),2)
   return(aDictValue)

def computeGSS(aDict1DF, aDict2DF, subCorpusSize1, subCorpusSize2, aSet=[]):
   aDictValue = {}
   acValue = float(subCorpusSize1)
   bdValue = float(subCorpusSize2)
   nValue = acValue + bdValue
   n2Value = nValue * nValue
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0001
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0.0001
      cValue = acValue - aValue
      dValue = bdValue - bValue
#     print aKey, aValue, bValue, cValue, dValue, (((aValue*dValue) - (cValue*bValue)) / n2Value)
      aDictValue [aKey] = ((aValue*dValue) - (cValue*bValue)) / n2Value
   return(aDictValue)

def computeOddRatio(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   acValue = float(corpusSize1)
   bdValue = float(corpusSize2)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0001
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0.0001
      cValue = acValue - aValue
      dValue = bdValue - bValue
      prob1 = aValue / acValue
      prob2 = bValue / bdValue
      if ((bValue <= 0) or (abs(aValue - acValue) <= 0.00000001)) :
         oddValue = 0.0
      else:
         oddValue = (prob1*(1-prob2)) / ((1-prob1)*prob2)
#      print aKey, aValue, bValue, cValue, dValue, oddValue
      aDictValue[aKey] = oddValue
   return(aDictValue)

def computeOddRatioOLD(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0001
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0.0001
      cValue = float(corpusSize1) - aValue
      dValue = float(corpusSize2) - bValue
      if ((cValue*bValue) <= 0):
         oddValue = 0.0
      else:
         oddValue = (aValue*dValue) / (cValue*bValue)
#      print aKey, aValue, bValue, cValue, dValue, oddValue
      aDictValue[aKey] = oddValue
   return(aDictValue)

# CHI Chi-square value  return a Dict (word:chi) from a corpusDFDict, subcorpusDFDict, 
def computeCHI(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   nValue = float(corpusSize1) + corpusSize2
   acValue = float(corpusSize1)
   bdValue = float(corpusSize2)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0
      abValue = aValue + bValue
      cValue = acValue - aValue
      dValue = bdValue - bValue
      num = nValue * pow( (float(aValue) * dValue)  - (cValue * bValue), 2)
      denum = float(abValue) * acValue * (bValue+dValue) * (cValue + dValue)
      if ((num <= 0) | (denum <= 0)):
         chiValue = 0.0
      else:
         chiValue =   num / denum
      aDictValue [aKey] = chiValue
#      print aKey, aValue, bValue, cValue, dValue, chiValue
   return(aDictValue)

# PMI Point Mutual Information   return a Dict (word:pmi) from a corpusDFDict, subcorpusDFDict, 
def computePMI(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   nValue = float(corpusSize1 + corpusSize2)
   acValue = float(corpusSize1)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0001
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0.0001
      abValue = aValue + bValue
      if (((abValue * acValue) <= 0) | (aValue <= 0)):
         pmiValue = 0.0
      else:
         pmiValue = math.log (  (float (aValue*nValue) / (abValue * acValue)), 2)
      aDictValue[aKey] = pmiValue
#      print aKey, aValue, bValue, pmiValue
   return(aDictValue)


# Gain Ratio value  return a Dict (word:chi) from a corpusDFDict, subcorpusDFDict, 
def computeGR(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   nValue  = float(corpusSize1) + corpusSize2
   acValue = float(corpusSize1)
   bdValue = float(corpusSize2)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0.0
      abValue = aValue + bValue
      cValue = acValue - aValue
      dValue = bdValue - bValue
      part1 = ((aValue) * nValue) / (abValue * acValue)
      part2 = (cValue * nValue) / (acValue * (cValue + dValue))
      if (aValue > 0):
         grValue =  (aValue / nValue) * math.log(part1,2)
      else:
         grValue = 0.0
      if (cValue > 0):
         grValue = grValue +  (cValue/nValue) * math.log(part2,2)
      aDictValue [aKey] = grValue
#      print aKey, aValue, bValue, cValue, dValue, grValue
   return(aDictValue)


# Information gain value  return a Dict (word:chi) from a corpusDFDict, subcorpusDFDict, 
def computeIG(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   nValue  = float(corpusSize1) + corpusSize2
   acValue = float(corpusSize1)
   bdValue = float(corpusSize2)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0.0
      abValue = aValue + bValue
      cValue  = acValue - aValue
      dValue  = bdValue - bValue

      if (aValue > 0):
         part1 = aValue / nValue
         part1 = part1 * math.log ((aValue * nValue) / (abValue * acValue))
      else:
         part1 = 0
      if (bValue > 0):        
         part2 = bValue / nValue
         part2 = part2 * math.log ((bValue * nValue) / (abValue * bdValue))
      else:
         part2 = 0
      if (cValue > 0):
         part3 = cValue / nValue
         part3 = part3 * math.log ((cValue * nValue) / (acValue * (cValue+dValue)))
      else:
         part3 = 0
      if (dValue > 0):
         part4 = dValue / nValue
         part4 = part4 * math.log ((dValue * nValue) / ((bdValue) * (cValue+dValue)))
      else:
         part4 = 0
      igValue = part1 + part2 + part3 + part4         
      aDictValue [aKey] = igValue
#      print aKey, aValue, bValue, cValue, dValue, igValue
   return(aDictValue)


def computeCC(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   nValue = float(corpusSize1) + corpusSize2
   acValue = float(corpusSize1)
   bdValue = float(corpusSize2)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = float(aDict2DF[aKey])
      else:
         bValue = 0.0
      if (aDict1DF.has_key(aKey)):
         aValue = float(aDict1DF[aKey])
      else:
         aValue = 0.0
      abValue = aValue + bValue
      cValue = acValue - aValue
      dValue = bdValue - bValue
      num = math.sqrt(nValue) * ((aValue * dValue)  - (cValue * bValue))
      denum = math.sqrt(abValue * acValue * bdValue * (cValue + dValue))
      if ((num <= 0) | (denum <= 0)):
         ccValue = 0.0
      else:
         ccValue =   num / denum
      aDictValue[aKey] = ccValue
#      print aKey, aValue, bValue, cValue, dValue, ccValue
   return(aDictValue)

# Relevance score  return a Dict 
def computeRS(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   nValue = float(corpusSize1 + corpusSize2)
   acValue = float(corpusSize1)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = aDict2DF[aKey]
      else:
         bValue = 0
      if (aDict1DF.has_key(aKey)):
         aValue = aDict1DF[aKey]
      else:
         aValue = 0
      cValue = acValue - aValue
      dValue = nValue - acValue - bValue
      num = (float(aValue) / acValue) + 1.0
      denum = (float(bValue) / (bValue + dValue)) + 1.0
      rsValue = math.log ( (num / denum), 2) 
      aDictValue [aKey] = rsValue
#      print aKey, aValue, bValue, cValue, dValue, rsValue
   return(aDictValue)


# WLLR Eighted Log LikelihoodRatio Measure  return a Dict (word:wllr) from a corpusDFDict, subcorpusDFDict, 
def computeWLLR(aDict1DF, aDict2DF, corpusSize1, corpusSize2, aSet=[]):
   aDictValue = {}
   nValue = float(corpusSize1 + corpusSize2)
   acValue = float(corpusSize1)
   if (len(aSet) == 0):
      aKeySet = union(aDict1DF.keys(), aDict2DF.keys())
   else:
      aKeySet = aSet
   for aKey in aKeySet:
      if (aDict2DF.has_key(aKey)):
         bValue = aDict2DF[aKey]
      else:
         bValue = 0.0001
      if (aDict1DF.has_key(aKey)):
         aValue = aDict1DF[aKey]
      else:
         aValue = 0
      cValue = acValue - aValue
      dValue = nValue - acValue - bValue
      num = float(aValue) / acValue
      denum = float(bValue) / (bValue + dValue)
      if (num <= 0) :
         wllrValue = -1.0
      elif (denum <= 0):
         wllrValue = 1.0 + (float(aValue) / nValue) 
      else:
         wllrValue =  num *  math.log ( (num / denum), 2) 
      aDictValue[aKey] = wllrValue
#      print aKey, aValue, bValue, cValue, dValue, wllrValue
   return(aDictValue)



#
# Read files 
#

def createDictionary(aFileName):
   myFile = codecs.open(aFileName, encoding='utf-8', mode="r")
   aDict = {}
   aNumber = 0
   aLine = myFile.readline()
   myLine = string.split(aLine)
   myLine = myLine[3:]
   while aLine:
      aNumber = aNumber + 1
      for aWord in myLine:
         if (aDict.has_key(aWord)):
            aValue = aDict[aWord]
         else:
            aValue = 0
         aDict[aWord] = aValue + 1
      aLine = myFile.readline()
      myLine = string.split(aLine)
      myLine = myLine[3:]
   myFile.close()
   return(aDict, aNumber)

def createDFDictionary(aFileName, startPos=3):
   myFile = codecs.open(aFileName, encoding='utf-8', mode="r")
   aDict = {}
   aNumber = 0
   aLine = myFile.readline()
   myLine = string.split(aLine)
   myLine = myLine[startPos:]
   while aLine:
      aDict1 = {}
      aNumber = aNumber + 1
      for aWord in myLine:
         aDict1[aWord] = 1
      for aWord in aDict1.keys():
         if (aDict.has_key(aWord)):
            aValue = aDict[aWord]
         else:
            aValue = 0
         aDict[aWord] = aValue + 1
      aLine = myFile.readline()
      myLine = string.split(aLine)
      myLine = myLine[startPos:]
   myFile.close()
   return(aDict, aNumber)


def readDictionary(aFileName):
   myFile = codecs.open(aFileName, encoding='utf-8', mode="r")
   aDict = {}
   aLine = myFile.readline()
   aLine = aLine.replace(u"\n", "")
   aPos = aLine.find(u':')
   aKey = aLine[:aPos]
   aValue = aLine[aPos+1:]
   while aLine:
      aDict[aValue] = float(aKey)
      aLine = myFile.readline()
      aLine = aLine.replace(u"\n", "")
      aPos = aLine.find(u':')
      aKey = aLine[:aPos]
      aValue = aLine[aPos+1:]
   myFile.close()
   return(aDict)




#
# Union and intersection
#

def intersection (*args):
   res = []
   for x in args[0]:
      for other in args[1:]:
         if x not in other: break
      else:
        res.append(x)
   return res

def union (*args):
   res = []
   for seq in args:
      for x in seq:
         if x not in res:
            res.append(x)
   return res

#
# Manipulating a dictionary
#

# remove entries for which the tf value <= minTF
def filterDictTF(aDict, minTF):
   for aKey in aDict.keys():
      if (aDict[aKey] <= minTF):
         del aDict[aKey]

# generate a new dict by normalizing the values (tf -> ntf)
# use to create a dict with relative frequencies
def reverseDict(aDict):
   aNewDict = {}
   for aKey in aDict.keys():
      aValue = aDict[aKey]
      if (aNewDict.has_key(aValue)):
         aList = aNewDict[aValue]
      else:
         aList = []
      aList.append(aKey)
      aNewDict[aValue] = aList
   return(aNewDict)

def dumpDictionary(aDictA, aFileName=""):
   aDict = reverseDict(aDictA)
   if (len(aFileName) > 0):
      myFile = codecs.open(aFileName, encoding='utf-8', mode="a")
   for aKey in sorted(aDict.keys(),None, None, True):
      aListKeyA = aDict[aKey]
      for aKeyA in aListKeyA:
         aValueA = aDictA[aKeyA]
         if (len(aFileName) > 0):
            myFile.write(str(aValueA)+":"+aKeyA+"\n")
         else:
            print str(aValueA)+"  :  "+aKeyA
   if (len(aFileName) > 0):
      myFile.close()
   return(aDict)

def extractTopSortedDict(aDict, aLimit, positiveOnly=False):
   if not(positiveOnly):
      return(extractTopSortedDictPrivate(aDict, aLimit))
   aNewDict = reverseDict(aDict)
   printed = 0
   aTopDict = {}
   for aKey in sorted(aNewDict.keys(),None,None,True):
      if (printed < aLimit):
         for aWord in aNewDict[aKey]:
            if (aKey < 0):
               break
            aTopDict[aWord] = aKey
            printed = printed + 1
            if (printed > aLimit):
               break
      else:
         break
   return(aTopDict)

def extractTopSortedDictPrivate(aDict, aLimit):
   aNewDict = reverseDict(aDict)
   printed = 0
   aTopDict = {}
   for aKey in sorted(aNewDict.keys(),None,None,True):
      if (printed < aLimit):
         for aWord in aNewDict[aKey]:
            aTopDict[aWord] = aKey
            printed = printed + 1
            if (printed > aLimit):
               break
      else:
         break
   return(aTopDict)


def extractBottomSortedDict(aDict, aLimit):
   aNewDict = reverseDict(aDict)
   printed = 0
   aBottomDict = {}
   for aKey in sorted(aNewDict.keys()):
      if (printed < aLimit):
         for aWord in aNewDict[aKey]:
            aBottomDict[aWord] = aKey
            printed = printed + 1
            if (printed > aLimit):
               break
      else:
         break
   return(aBottomDict)

# print only when not imported
if __name__ == '__main__':
   print "The module TermSelection.py was uploaded\n"
